## Import packages and initialize directories

In [1]:
import glob
import pandas as pd
import pickle
import scipy.io as sio
from scipy.io import loadmat
import unittest
import os
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib

import mne
from scipy import stats
from scipy import signal
import time

import warnings
warnings.filterwarnings('ignore')

beh_dir = '../../data/decision-making/data/data_behav'
neur_dir = '../../data/decision-making/data/data_ephys'
preproc_dir = '../../data/decision-making/data/data_preproc'

beh_files = [file for file in glob.glob(os.path.join(beh_dir,"gamble.data*.csv"))]
neur_files = [file for file in glob.glob(os.path.join(neur_dir,"*.mat"))]

sfreq = 1000

In [2]:
# turn off interactive mode to use plt.psd just to get data, not to plot PSD for each trial
plt.ioff()

In [3]:
sample_first = 0
sample_last = 950
num_samples = sample_last-sample_first

## Create df where each row corresponds to 1 subject-electrode-trial = 1 example

It looks like there is meaningful variation in power up to 50 Hz, and after that, it's qualitatively different, probably noise. I will extract analytic amplitude in 2 Hz bins up to 50 Hz, and use as features. See 0-preprocess-explore notebook for the images that generated this conclusion.

In [4]:
# read bad_trials, for exclusion
bad_trials = sio.loadmat(os.path.join(beh_dir, 'bad_trials_OFC.mat'))['bad_trials_OFC']

# read game_model which, we hope, is identical across subjects
game_model = pd.read_csv(os.path.join(beh_dir,'gamble_choices.csv'))

# make master df where you append each subject's df
df_master = pd.DataFrame(columns = ['subject', 
                                    'include.trial', 
                                    'round', 
                                    'newround.time', 
                                    'choice.time',
                                    'buttonpress.time', 
                                    'conf.time', 
                                    'reveal.time', 
                                    'choice.class',
                                    'choice.location', 
                                    'outcome', 
                                    'Safe.Bet.Amount', 
                                    'Risky.Bet.Amount',
                                    'Risky.bet.shown.number', 
                                    'Risky.bet.hidden.number', 
                                    'Risky.Side',
                                    'data', 
                                    'channel'])

# Extract filtered data in 2-Hz bins from 0 to 50 Hz
center_frequencies = np.linspace(1,49,num=25,dtype='int')

for sub_index, files in enumerate(zip(beh_files, neur_files)):
    beh_file = files[0]
    neur_file = files[1]
    
    print(sub_index)
    print()
    
    ## Read data
    # ------------------------------------------------------------------------------------------------------.
    # behavior
    df = pd.DataFrame()
    df = pd.read_csv(os.path.join(beh_file))
    
    # neural
    neur = loadmat(neur_file)['buttonpress_events_hg']
    
    ## Number trials and number electrodes
    # ------------------------------------------------------------------------------------------------------.
    num_trials_beh = len(df)
    
    num_trials = neur.shape[0]
    num_samples = neur.shape[1]
    nchan = neur.shape[2]
    
    # add subject column on the left: make it be 1-indexed, corresponding to the subid's in the file
    df.insert(0, 'subject', sub_index+1)
    
    ## Append game model data
    # ------------------------------------------------------------------------------------------------------.
    df = df.merge(game_model[:num_trials_beh], left_index=True, right_index=True)    

    ## Exclude bad trials from entire df: Makes it easier to match with neural data
    # ------------------------------------------------------------------------------------------------------.
    df.insert(1, 'include.trial', (bad_trials[sub_index,:num_trials_beh]==0) & (df['choice.location']!='Timeout'))
    # exclude trials (shorten df)
    df = df[df['include.trial']]
    
    # create a new index that just counts up to the number of included trials, and corresponds to the neural data
    df.insert(0, 'trial_index_subject', np.arange(num_trials))
    df = df.set_index('trial_index_subject')
    
    ## Add neural data
    # ------------------------------------------------------------------------------------------------------.
    # initialize a data column, that will take a row of data subject-electrode-trial,
    # so a 1d-array of the number of time points in the data
    df = df.assign(data=None)
    df = df.assign(channel=None)
    df_subject = pd.DataFrame(columns = df.columns)
    
    # loop over electrodes
    for this_chan in range(nchan):
        # create a dataframe for this specific channel, containing the behavior data for this subject
        df_chan = df.copy()
        df_chan['channel'] = this_chan
        # loop over trials
        for this_trial in range(num_trials):
            # insert data for each trial of df: the neural data for electrode 0, that trial
            # FIRST 950 time points only, in format that can be tf-decomposed
            data = neur[this_trial,:sample_last,this_chan].astype('float64').reshape(1,-1)
            # Get TF-features, analytic amplitude at each time point, in 2 Hz-bins from 0 to 50 Hz
            # initialize tf-array
#             tf = np.zeros([len(center_frequencies), 950])
#             for index, center_frequency in enumerate(center_frequencies):
#                 # compute analytic amplitude using filter-hilbert method
#                 # z-score within frequency bin
#                 tf[index,:] = stats.zscore(np.abs(signal.hilbert(mne.filter.filter_data(data, sfreq, center_frequency-1, center_frequency+1)[0])))
#             pass
#             # include both the "raw" data and the flattened tf-representation
#             df_chan.at[this_trial, 'data'] = list(np.hstack([data.flatten(),tf.flatten()]))
            df_chan.at[this_trial, 'data'] = list(data.flatten())
        df_subject = df_subject.append(df_chan)
    df_master = df_master.append(df_subject)
    
    df_master.insert(0, 'index', np.arange(len(df_master)))
    df_master = df_master.set_index('index')

0

1

2

3

4

5

6

7

8

9



## Include only gambles where first number is 5 or 6

In [5]:
df_use = df_master[(df_master['Risky.bet.shown.number']==5) | (df_master['Risky.bet.shown.number']==6)]
df_use.insert(0, 'index_use', np.arange(len(df_use)))
df_use = df_use.set_index('index_use')

In [6]:
np.mean(df_use['choice.class']=='Gamble')

0.5698814409484724

In [7]:
len(df_use)

8772

## Insert frequency with maximal amplitude value as feature (takes a while to compute, so do it only for included gambles)

In [8]:
for this_index in df_use.index:
    if this_index%200==0:
        print(this_index)
    data = df_use.data[this_index]
    this_psd = plt.psd(np.asarray(data), Fs=sfreq);
    xfreqs = this_psd[1]
    yvals = np.log(this_psd[0])
    freq_with_max_amp = xfreqs[np.argmax(yvals)]
    max_amp_in_freq = yvals.max()
    df_use.at[this_index,'data'] = list(np.hstack([data,freq_with_max_amp,max_amp_in_freq,yvals[:50]]))

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600


## Extract a *separate* X-matrix, and separate y-labels, for each subject

In [9]:
# make sure this is 951
len(df_use.data[this_index])

1002

In [10]:
num_examples = len(df_use)
num_samples = len(df_use.data[this_index])
# # extract all the listed data into an array
X = np.empty([num_examples,num_samples])

for this_example in range(num_examples):
    X[this_example,:] = np.asarray(df_use['data'][this_example])

### Check here that X has the right shape

Loop over subjects and store in dictionary

In [11]:
subjects = set(df_use['subject'])

Xdict = dict()
ydict = dict()

for this_subject in subjects:
#     print(this_subject)
    df_subject = df_use[df_use['subject']==this_subject]
    num_examples = len(df_subject)
    X = np.empty([num_examples,num_samples])

    for this_example in range(num_examples):
#         X[this_example,sample_first:sample_last] = np.asarray(df_use['data'][this_example])[sample_first:sample_last]
        X[this_example,:] = np.asarray(df_use['data'][this_example])
#     print(X.shape)
    Xdict[this_subject] = X
    ydict[this_subject] = df_subject['choice.class'].values
#     print(len(ydict[this_subject]))

### Check here that contents of Xdict and ydict have the right shapes

## Save X and y dictionaries (took a while to compute, so important to save)

In [12]:
import pickle

In [13]:
# !mkdir data

In [14]:
with open(os.path.join('data','Xdict.pickle'), 'wb') as handle1:
    pickle.dump(Xdict, handle1, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
with open(os.path.join('data','ydict.pickle'), 'wb') as handle2:
    pickle.dump(ydict, handle2, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
os.listdir('data')

['Xdict.pickle', 'ydict.pickle']